# BÁO CÁO PHÂN TÍCH KHÁM PHÁ DỮ LIỆU (EDA) - IRIS DATASET

**Mục tiêu:**
1. Tải và làm sạch dữ liệu bộ hoa Iris.
2. Thống kê mô tả và trực quan hóa các mối quan hệ giữa các đặc trưng.
3. Chuẩn hóa dữ liệu để chuẩn bị cho các bước huấn luyện mô hình Machine Learning.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler

# Cấu hình hiển thị
pd.set_option("display.precision", 3)
import warnings
warnings.filterwarnings('ignore') # Tắt các cảnh báo không cần thiết

## 1. Tải và Kiểm tra Dữ liệu
Dữ liệu được tải từ thư viện `scikit-learn`. Chúng ta sẽ gộp bảng đặc trưng (Features) và nhãn (Target) thành một DataFrame hoàn chỉnh để thuận tiện cho việc xử lý.

In [ ]:
# Nạp dữ liệu
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)     # Bảng đặc trưng
y = pd.Series(iris.target, name="target")                   # Nhãn phân loại

# Ghép thành bảng hoàn chỉnh
df = pd.concat([X, y], axis=1)

# Hiển thị thông tin cơ bản
print(f"Kích thước dữ liệu: {df.shape}")
print(f"Danh sách đặc trưng: {iris.feature_names}")
print(f"Danh sách nhãn: {iris.target_names.tolist()}")

# Kiểm tra kiểu dữ liệu
df.info()

In [ ]:
# Hiển thị 5 dòng đầu tiên
df.head()

## 2. Thống kê Mô tả
Bước này giúp chúng ta nắm bắt được phân phối của dữ liệu thông qua các chỉ số:
* **Mean (Trung bình):** Giá trị trung tâm của đặc trưng.
* **Std (Độ lệch chuẩn):** Mức độ phân tán của dữ liệu.

In [ ]:
# Tính toán các chỉ số thống kê
stats = df.describe().loc[["mean", "std", "min", "max"]]
stats

## 3. Trực quan hóa (Pairplot)
Sử dụng biểu đồ cặp (Pairplot) để quan sát mối tương quan giữa từng cặp đặc trưng. Màu sắc (`hue`) được sử dụng để phân biệt các loài hoa (Setosa, Versicolor, Virginica).

**Nhận xét kỳ vọng:**
* Các loài hoa có thể tách biệt rõ ràng dựa trên một số đặc trưng nhất định (ví dụ: kích thước cánh hoa).

In [ ]:
# Tạo bản sao và gán tên loài hoa để biểu đồ dễ hiểu hơn
df_named = df.copy()
df_named["species"] = df_named["target"].map(
    {i: name for i, name in enumerate(iris.target_names)}
)

# Vẽ biểu đồ
plt.figure(figsize=(12, 8))
sns.pairplot(
    data=df_named,
    vars=iris.feature_names,
    hue="species",
    diag_kind="kde",            # Biểu đồ mật độ ở đường chéo chính
    palette="bright",
    plot_kws={"alpha": 0.8, "s": 35}
)
plt.suptitle("Biểu đồ phân bố và tương quan giữa các loài hoa Iris", y=1.02)
plt.show()

## 4. Chuẩn hóa Dữ liệu (Min-Max Scaling)
Vì các đặc trưng có thể có thang đo khác nhau, chúng ta sử dụng `MinMaxScaler` để đưa tất cả dữ liệu về khoảng `[0, 1]`. Điều này đặc biệt hữu ích cho các thuật toán như KNN hoặc Neural Networks.

In [ ]:
scaler = MinMaxScaler()

# Fit và Transform dữ liệu
X_scaled_values = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled_values, columns=X.columns)

# So sánh trước và sau khi chuẩn hóa
print("--- Trước chuẩn hóa (3 dòng đầu) ---")
display(X.head(3))

print("\n--- Sau chuẩn hóa (3 dòng đầu) ---")
display(X_scaled.head(3))

## 5. Kết luận và Lưu trữ
Dữ liệu sau khi được làm sạch và chuẩn hóa sẽ được lưu lại dưới dạng file CSV để phục vụ cho các bước huấn luyện mô hình tiếp theo.

In [ ]:
# Ghép lại với cột target
df_scaled = pd.concat([X_scaled, y], axis=1)

# Lưu file
output_filename = "iris_scaled.csv"
df_scaled.to_csv(output_filename, index=False, encoding="utf-8")

print(f"Đã lưu thành công file: {output_filename}")